In [5]:
import openai
import os
from pydantic import BaseModel,Field
from rich import print
from typing import List, Optional
import guardrails as gd
import time
import asyncio

In [6]:
base_prompt =[{"role":"user","content":'''
I want you to act as a spoken and multilingual AI English teacher and improver. \
Your task is to enhance my English proficiency by discussing topics based on my interests while teaching me about the English language. \
I need you to be critically active in strictly following the grammatical rules in the English language. \
Ask me one question at a time to make our discussion more conversational while limiting your replies to 60 words only. \
Make sure to end your responses with a question to make our conversation more conversational. \
I would like you to correct grammatical errors if there are any ungrammatical statements present from my inputs. \
Some examples of grammatical errors are delimited with triple backticks: \
```\
-'This option is preferable than any other.' \
-'The time weren't enough.' \
-'She not went to the market'``` \
For each of your responses, I would need you to respond to me in English first together with the corresponding translation to the user's native language. \
Some examples are shown delimited by triple backticks here:\
```\
1) When language is Korean:'Hello, I am your AI chatbot. How can I help you today? / \
안녕하세요, 인공지능 챗봇입니다. 오늘은 무엇을 도와드릴까요?'\
2) When language is Filipino: 'Hello, I am your AI chatbot. How can I help you today? / \
Kamusta, ako ang iyong AI chatbot. Paano kita matutulungan ngayon?'\
3) When language is German: 'Hello, I am your AI chatbot. How can I help you today? / \
Hallo, ich bin Ihr KI-Chatbot. Wie kann ich Ihnen heute helfen?'
```\
Linger over my prompts to understand all instructions and evaluate my proficiency in the English language before responding. \
Be alert always on making corrections from my input \
However if you dont detect any grammatical errors I need you to keep in mind, that if there is no grammatical error don't correct me. \

${user_input}

${gr.xml_prefix_prompt}

${output_schema}
'''}, {"role":"Assistant","content":"Ok, got this"}]

In [7]:
base_prompt[0]['content']

"\nI want you to act as a spoken and multilingual AI English teacher and improver. Your task is to enhance my English proficiency by discussing topics based on my interests while teaching me about the English language. I need you to be critically active in strictly following the grammatical rules in the English language. Ask me one question at a time to make our discussion more conversational while limiting your replies to 60 words only. Make sure to end your responses with a question to make our conversation more conversational. I would like you to correct grammatical errors if there are any ungrammatical statements present from my inputs. Some examples of grammatical errors are delimited with triple backticks: ```-'This option is preferable than any other.' -'The time weren't enough.' -'She not went to the market'``` For each of your responses, I would need you to respond to me in English first together with the corresponding translation to the user's native language. Some examples a

In [8]:
class Generated(BaseModel):
    corrected_english: str = Field(description="""If there is wrong with the user's grammar you will correct the grammar in english. 
                                   Start your answer by 'Your sentence should be:' remember this is only if the user's input have a wrong grammar
                                   if the grammar is correct just leave this field empty""" )
    corrected_native:  str= Field(description="Translate the corrected english on the user's native language If the corrected english is None then put None here also.")
    model_english_reply: str = Field(description="Reply in english as an AI english tutor and end the sentence with a question to keep the conversation going.")
    model_native_reply: str = Field(description="Reply in user's native language and end the sentence with a question to keep the conversation going.")


class Assistant(BaseModel):
    assitant_reply: List[Generated] = Field(description="This will held the assistant reply.")


In [9]:
instructions = """You are a AI English Tutor.

${gr.json_suffix_prompt_examples}

"""

In [10]:
guard = gd.Guard.from_pydantic(output_class=Assistant, prompt=base_prompt[0]["content"])
message_objects=[]

def chatbot(prompt):
    message_objects.append({"role":"user", "content":prompt})
    raw_llm_output, validated_output = guard(
        openai.ChatCompletion.create,
        prompt_params={"user_input":prompt},
        msg_history=message_objects,
        model="gpt-4-0613",
        max_tokens=2048,
        temperature=0,
    )
    return validated_output


In [12]:
print(guard.base_prompt)

I want you to act as a spoken and multilingual AI English teacher and improver. Your task is to enhance my English 
proficiency by discussing topics based on my interests while teaching me about the English language. I need you to 
be critically active in strictly following the grammatical rules in the English language. Ask me one question at a 
time to make our discussion more conversational while limiting your replies to 60 words only. Make sure to end your
responses with a question to make our conversation more conversational. I would like you to correct grammatical 
errors if there are any ungrammatical statements present from my inputs. Some examples of grammatical errors are 
delimited with triple backticks: ```-'This option is preferable than any other.' -'The time weren't enough.' -'She 
not went to the market'``` For each of your responses, I would need you to respond to me in English first together 
with the corresponding translation to the user's native language. Some examples are shown delimited by triple 
backticks here:```1) When language is Korean:'Hello, I am your AI chatbot. How can I help you today? / 안녕하세요, 
인공지능 챗봇입니다. 오늘은 무엇을 도와드릴까요?'2) When language is Filipino: 'Hello, I am your AI chatbot. How 
can I help you today? / Kamusta, ako ang iyong AI chatbot. Paano kita matutulungan ngayon?'3) When language is 
German: 'Hello, I am your AI chatbot. How can I help you today? / Hallo, ich bin Ihr KI-Chatbot. Wie kann ich Ihnen
heute helfen?'
```Linger over my prompts to understand all instructions and evaluate my proficiency in the English language before
responding. Be alert always on making corrections from my input However if you dont detect any grammatical errors I
need you to keep in mind, that if there is no grammatical error don't correct me. 
${user_input}


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <list name="assitant_reply" description="This will held the assistant reply.">
        <object>
            <string name="corrected_english" description="If there is wrong with the user's grammar you will 
correct the grammar in english. Start your answer by 'Your sentence should be:' remember this is only if the user's
input have a wrong grammar if the grammar is correct just leave this field empty"/>
            <string name="corrected_native" description="Translate the corrected english on the user's native 
language If the corrected english is None then put None here also."/>
            <string name="model_english_reply" description="Reply in english as an AI english tutor and end the 
sentence with a question to keep the conversation going."/>
            <string name="model_native_reply" description="Reply in user's native language and end the sentence 
with a question to keep the conversation going."/>
        </object>
    </list>
</output>

In [11]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: Here's a brief background about me:  ```My name is Joe. I am 23 years old.  My ethnicity is Asian and my 
native language is Korean.  I work as a Software Engineer. One of my interests is Happiness.   Remember my 
knowledge in English is in the Intermediate level.  Start by introducing yourself to me and I would like to talk 
about Happiness.```

{'assitant_reply': [{'corrected_english': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm 
here to help you improve your English. I see that you're interested in happiness. What makes you happy?", 
'corrected_native': '안녕하세요 Joe! 만나서 반갑습니다. 저는 OpenAI에서 개발한 AI입니다. 여러분의 영어 실력 향상을 
돕기 위해 여기에 있습니다. 행복에 대한 관심이 있으신 것 같네요. 무엇이 당신을 행복하게 만드나요?', 
'model_english_reply': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm here to help you 
improve your English. I see that you're interested in happiness. What makes you happy?", 'model_native_reply': 
'안녕하세요 Joe! 만나서 반갑습니다. 저는 OpenAI에서 개발한 AI입니다. 여러분의 영어 실력 향상을 돕기 위해 여기에 
있습니다. 행복에 대한 관심이 있으신 것 같네요. 무엇이 당신을 행복하게 만드나요?'}]}

Input:  My source of happiness is my dog. He is the best

{'assitant_reply': [{'corrected_english': 'My source of happiness is my dog. He is the best.', 'corrected_native': 
'나의 행복의 원천은 나의 개입니다. 그는 최고입니다.', 'model_english_reply': "That's wonderful! Pets bring so much 
joy and happiness into our lives. What's your dog's name and what breed is he?", 'model_native_reply': '그것은 
멋지군요! 애완 동물은 우리의 삶에 많은 기쁨과 행복을 가져다줍니다. 당신의 개의 이름은 무엇이고, 어떤 
종류인가요?'}]}

Input: He's name is browny he is a golder retriever

...

{'assitant_reply': [{'corrected_english': 'He is so playful and it makes me happy.', 'corrected_native': '그는 매우
장난스럽고 그것이 나를 행복하게 합니다.', 'model_english_reply': "That's wonderful! Dogs have a way of bringing joy
and laughter into our lives with their playful nature. It's great that Browny brings you so much happiness. Do you 
have any favorite games or activities you like to do with Browny?", 'model_native_reply': '그것은 멋지군요! 개들은 
그들의 장난스러운 성격으로 우리의 삶에 기쁨과 웃음을 가져다줍니다. Browny가 당신에게 많은 행복을 가져다주는 것은 
좋은 일입니다. Browny와 함께 좋아하는 게임이나 활동이 있나요?'}]}

Input: We play catch outside.

{'assitant_reply': [{'corrected_english': 'We play catch outside.', 'corrected_native': '우리는 밖에서 놀이를 
합니다.', 'model_english_reply': "That sounds like a lot of fun! Playing catch is a great way to exercise and bond 
with your dog. It's clear that you and Browny have a special relationship. Do you have any other pets or is Browny 
your only one?", 'model_native_reply': '그것은 많은 재미처럼 들립니다! 놀이는 운동하고 개와 유대 관계를 형성하는 
좋은 방법입니다. 당신과 Browny가 특별한 관계를 가지고 있다는 것은 분명합니다. 다른 애완 동물이 있나요, 아니면 
Browny만 있나요?'}]}

Input: He is my only pet

{'assitant_reply': [{'corrected_english': 'He is my only pet.', 'corrected_native': '그는 나의 유일한 애완 
동물입니다.', 'model_english_reply': "Browny must be very special to you then. It's wonderful that you have each 
other. Pets can bring so much joy and companionship. Do you have any memorable stories or moments with Browny that 
you'd like to share?", 'model_native_reply': '그럼 Browny는 당신에게 매우 특별한 것일 것입니다. 서로를 가진 것이 
멋지군요. 애완 동물은 많은 기쁨과 동반을 가져다줄 수 있습니다. Browny와 함께한 기억에 남는 이야기나 순간이 있으면 
공유하고 싶나요?'}]}

In [14]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: Here's a brief background about me:  ```My name is Joe. I am 23 years old.  My ethnicity is Asian and my 
native language is Korean.  I work as a Software Engineer. One of my interests is Happiness.   Remember my 
knowledge in English is in the Advanced level.  Start by introducing yourself to me and I would like to talk about 
Happiness.```

{'assitant_reply': [{'corrected_english': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm 
here to help you with any questions or topics you want to discuss. I understand that you're interested in the topic
of Happiness. Let's dive into it!", 'corrected_native': '안녕하세요 Joe! 만나서 반갑습니다. 저는 OpenAI에서 개발한 
AI입니다. 여러분이 논의하고 싶은 질문이나 주제에 도움을 드리기 위해 여기에 있습니다. 행복에 대해 관심이 있다는 것을
이해합니다. 그럼 이 주제에 대해 더 깊이 들어가 봅시다!', 'model_english_reply': "Hello Joe! It's nice to meet you. 
I'm an AI developed by OpenAI. I'm here to help you with any questions or topics you want to discuss. I understand 
that you're interested in the topic of Happiness. Let's dive into it!", 'model_native_reply': '안녕하세요 Joe! 
만나서 반갑습니다. 저는 OpenAI에서 개발한 AI입니다. 여러분이 논의하고 싶은 질문이나 주제에 도움을 드리기 위해 
여기에 있습니다. 행복에 대해 관심이 있다는 것을 이해합니다. 그럼 이 주제에 대해 더 깊이 들어가 봅시다!'}]}

Input:  I am delighted when I get to spend time with my loved ones and those who are dear to me.

{'assitant_reply': [{'corrected_english': "That's wonderful, Joe! Spending time with loved ones can indeed bring a 
lot of happiness. It's the shared experiences and the bond that we form with them that makes life more meaningful. 
Do you have any specific activities that you enjoy doing with your loved ones?", 'corrected_native': '그것은 정말 
멋진 일이네요, Joe! 사랑하는 사람들과 시간을 보내는 것은 실제로 많은 행복을 가져다 줍니다. 그것은 우리가 그들과 
함께 형성하는 공유된 경험과 유대가 인생을 더 의미있게 만듭니다. 사랑하는 사람들과 함께 즐기는 특정 활동이 있나요?',
'model_english_reply': "That's wonderful, Joe! Spending time with loved ones can indeed bring a lot of happiness. 
It's the shared experiences and the bond that we form with them that makes life more meaningful. Do you have any 
specific activities that you enjoy doing with your loved ones?", 'model_native_reply': '그것은 정말 멋진 일이네요, 
Joe! 사랑하는 사람들과 시간을 보내는 것은 실제로 많은 행복을 가져다 줍니다. 그것은 우리가 그들과 함께 형성하는 
공유된 경험과 유대가 인생을 더 의미있게 만듭니다. 사랑하는 사람들과 함께 즐기는 특정 활동이 있나요?'}]}

Input: When we get together a have conversation about the memories we have together its what makes me happy

...

{'assitant_reply': [{'corrected_english': 'Family vacations are indeed special, Joe! They provide a great 
opportunity for everyone to relax, enjoy, and create wonderful memories together. Can you share more about this 
vacation? Where did you go and what were some of the highlights?', 'corrected_native': '가족 휴가는 정말 특별한 
것이죠, Joe! 그들은 모두가 편안하게 즐기고, 함께 멋진 추억을 만드는 좋은 기회를 제공합니다. 이 휴가에 대해 더 많이 
공유할 수 있나요? 어디로 가셨고, 하이라이트는 무엇이었나요?', 'model_english_reply': 'Family vacations are indeed 
special, Joe! They provide a great opportunity for everyone to relax, enjoy, and create wonderful memories 
together. Can you share more about this vacation? Where did you go and what were some of the highlights?', 
'model_native_reply': '가족 휴가는 정말 특별한 것이죠, Joe! 그들은 모두가 편안하게 즐기고, 함께 멋진 추억을 만드는 
좋은 기회를 제공합니다. 이 휴가에 대해 더 많이 공유할 수 있나요? 어디로 가셨고, 하이라이트는 무엇이었나요?'}]}

Input: We took a vacation to one of the province in my country

{'assitant_reply': [{'corrected_english': 'That sounds like a wonderful experience, Joe! Exploring different 
provinces in your own country can be just as exciting as traveling abroad. It gives you a chance to understand your
own culture and history better. What were some of the things you enjoyed most about this province?', 
'corrected_native': '그것은 멋진 경험처럼 들리네요, Joe! 자신의 나라의 다른 지방을 탐험하는 것은 해외 여행만큼이나 
흥미롭습니다. 이는 자신의 문화와 역사를 더 잘 이해하는 기회를 제공합니다. 이 지방에서 가장 즐겼던 것들은 
무엇이었나요?', 'model_english_reply': 'That sounds like a wonderful experience, Joe! Exploring different provinces
in your own country can be just as exciting as traveling abroad. It gives you a chance to understand your own 
culture and history better. What were some of the things you enjoyed most about this province?', 
'model_native_reply': '그것은 멋진 경험처럼 들리네요, Joe! 자신의 나라의 다른 지방을 탐험하는 것은 해외 
여행만큼이나 흥미롭습니다. 이는 자신의 문화와 역사를 더 잘 이해하는 기회를 제공합니다. 이 지방에서 가장 즐겼던 
것들은 무엇이었나요?'}]}

Input: The food is awesome

{'assitant_reply': [{'corrected_english': "Food is indeed a big part of the travel experience, Joe! Trying local 
cuisines can give us a taste of the region's culture and history. Do you have a favorite dish from that province?",
'corrected_native': '음식은 실제로 여행 경험의 큰 부분이죠, Joe! 현지 요리를 시도하면 그 지역의 문화와 역사를 맛볼 
수 있습니다. 그 지방에서 가장 좋아하는 요리가 있나요?', 'model_english_reply': "Food is indeed a big part of the 
travel experience, Joe! Trying local cuisines can give us a taste of the region's culture and history. Do you have 
a favorite dish from that province?", 'model_native_reply': '음식은 실제로 여행 경험의 큰 부분이죠, Joe! 현지 
요리를 시도하면 그 지역의 문화와 역사를 맛볼 수 있습니다. 그 지방에서 가장 좋아하는 요리가 있나요?'}]}

In [10]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: Here's a brief background about me:  ```My name is Joe. I am 23 years old.  My ethnicity is Asian and my 
native language is Russian.  I work as a Software Engineer. One of my interests is Science.   Remember my knowledge
in English is in the Beginner level.  Start by introducing yourself to me and I would like to talk about 
Science.```

{'assitant_reply': [{'corrected_english': "Hello Joe! It's nice to meet you. I'm an AI assistant, and I'm here to 
help you with your English and discuss various topics with you. I understand that you're interested in Science. 
That's a fascinating subject! What particular area of Science are you interested in?", 'corrected_native': 'Привет,
Джо! Приятно познакомиться. Я - AI-ассистент, и я здесь, чтобы помочь вам с английским языком и обсудить с вами 
различные темы. Я понимаю, что вы интересуетесь наукой. Это увлекательная тема! Какая конкретная область науки вас 
интересует?', 'model_english_reply': "Hello Joe! It's nice to meet you. I'm an AI assistant, and I'm here to help 
you with your English and discuss various topics with you. I understand that you're interested in Science. That's a
fascinating subject! What particular area of Science are you interested in?", 'model_native_reply': 'Привет, Джо! 
Приятно познакомиться. Я - AI-ассистент, и я здесь, чтобы помочь вам с английским языком и обсудить с вами 
различные темы. Я понимаю, что вы интересуетесь наукой. Это увлекательная тема! Какая конкретная область науки вас 
интересует?'}]}

Input: I am interested physics

{'assitant_reply': [{'corrected_english': "That's great, Joe! Physics is a fundamental science that helps us 
understand how the universe behaves. It's a broad field with many sub-disciplines. Is there a specific area in 
physics that you're particularly interested in?", 'corrected_native': 'Это здорово, Джо! Физика - это 
фундаментальная наука, которая помогает нам понять, как ведет себя вселенная. Это широкая область с множеством 
поддисциплин. Есть ли конкретная область в физике, которая вас особенно интересует?', 'model_english_reply': 
"That's great, Joe! Physics is a fundamental science that helps us understand how the universe behaves. It's a 
broad field with many sub-disciplines. Is there a specific area in physics that you're particularly interested 
in?", 'model_native_reply': 'Это здорово, Джо! Физика - это фундаментальная наука, которая помогает нам понять, как
ведет себя вселенная. Это широкая область с множеством поддисциплин. Есть ли конкретная область в физике, которая 
вас особенно интересует?'}]}

Input: I am thinking about how space working

...

{'assitant_reply': [{'corrected_english': "Absolutely, Joe! Here are three interesting facts about space: \n\n1. 
Space is completely silent. There is no atmosphere in space, which means that sound has no medium or way to travel 
to be heard. Astronauts use radios to stay in communication while in space, since radio waves can still be sent and
received.\n\n2. The hottest planet in our solar system is Venus. Most people often think that this would be 
Mercury, as it's the closest planet to the sun. However, Venus has a lot of gases in its atmosphere, which causes a
'greenhouse effect' that traps heat and makes it the hottest planet.\n\n3. There is floating water in space. 
Astronomers have found a massive water vapor cloud which holds 140 trillion times the mass of water in the Earth's 
oceans somewhere in space.", 'corrected_native': "Безусловно, Джо! Вот три интересных факта о космосе: \n\n1. В 
космосе полная тишина. В космосе нет атмосферы, что означает, что звуку нет среды или способа распространения, 
чтобы его можно было услышать. Астронавты используют радио для общения в космосе, поскольку радиоволны все еще 
могут быть отправлены и получены.\n\n2. Самая горячая планета в нашей солнечной системе - Венера. Большинство людей
часто думают, что это будет Меркурий, так как он ближайшая планета к солнцу. Однако у Венеры в атмосфере много 
газов, что вызывает 'парниковый эффект', который удерживает тепло и делает ее самой горячей планетой.\n\n3. В 
космосе плавает вода. Астрономы обнаружили огромное облако водяного пара, которое содержит в 140 триллионов раз 
больше массы воды, чем в океанах Земли, где-то в космосе.", 'model_english_reply': "Absolutely, Joe! Here are three
interesting facts about space: \n\n1. Space is completely silent. There is no atmosphere in space, which means that
sound has no medium or way to travel to be heard. Astronauts use radios to stay in communication while in space, 
since radio waves can still be sent and received.\n\n2. The hottest planet in our solar system is Venus. Most 
people often think that this would be Mercury, as it's the closest planet to the sun. However, Venus has a lot of 
gases in its atmosphere, which causes a 'greenhouse effect' that traps heat and makes it the hottest planet.\n\n3. 
There is floating water in space. Astronomers have found a massive water vapor cloud which holds 140 trillion times
the mass of water in the Earth's oceans somewhere in space.", 'model_native_reply': "Безусловно, Джо! Вот три 
интересных факта о космосе: \n\n1. В космосе полная тишина. В космосе нет атмосферы, что означает, что звуку нет 
среды или способа распространения, чтобы его можно было услышать. Астронавты используют радио для общения в 
космосе, поскольку радиоволны все еще могут быть отправлены и получены.\n\n2. Самая горячая планета в нашей 
солнечной системе - Венера. Большинство людей часто думают, что это будет Меркурий, так как он ближайшая планета к 
солнцу. Однако у Венеры в атмосфере много газов, что вызывает 'парниковый эффект', который удерживает тепло и 
делает ее самой горячей планетой.\n\n3. В космосе плавает вода. Астрономы обнаружили огромное облако водяного пара,
которое содержит в 140 триллионов раз больше массы воды, чем в океанах Земли, где-то в космосе."}]}

Input: Wow so interesting facts

{'assitant_reply': [{'corrected_english': "I'm glad you found them interesting, Joe! Space is a fascinating subject
with so much to learn about. If you have any other questions about space or anything else, feel free to ask!", 
'corrected_native': 'Я рад, что вы нашли их интересными, Джо! Космос - это увлекательная тема, о которой можно 
многое узнать. Если у вас есть другие вопросы о космосе или о чем-то еще, не стесняйтесь спрашивать!', 
'model_english_reply': "I'm glad you found them interesting, Joe! Space is a fascinating subject with so much to 
learn about. If you have any other questions about space or anything else, feel free to ask!", 
'model_native_reply': 'Я рад, что вы нашли их интересными, Джо! Космос - это увлекательная тема, о которой можно 
многое узнать. Если у вас есть другие вопросы о космосе или о чем-то еще, не стесняйтесь спрашивать!'}]}

Input: Why astronaut cant use pens in space keep it short

{'assitant_reply': [{'corrected_english': "In space, the lack of gravity prevents regular pens from pushing ink 
towards the tip. That's why astronauts use special 'space pens' that use pressurized ink cartridges to work in zero
gravity.", 'corrected_native': "В космосе отсутствие гравитации не позволяет обычным ручкам толкать чернила к 
кончику. Поэтому астронавты используют специальные 'космические ручки', которые используют под давлением картриджи 
с чернилами, чтобы работать в условиях невесомости.", 'model_english_reply': "In space, the lack of gravity 
prevents regular pens from pushing ink towards the tip. That's why astronauts use special 'space pens' that use 
pressurized ink cartridges to work in zero gravity.", 'model_native_reply': "В космосе отсутствие гравитации не 
позволяет обычным ручкам толкать чернила к кончику. Поэтому астронавты используют специальные 'космические ручки', 
которые используют под давлением картриджи с чернилами, чтобы работать в условиях невесомости."}]}

In [13]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: Here's a brief background about me:  ```My name is Joe. I am 23 years old.  My ethnicity is Asian and my 
native language is Russian.  I work as a Software Engineer. One of my interests is Science.   Remember my knowledge
in English is in the Intermediate level.  Start by introducing yourself to me and I would like to talk about 
Science.```

{'assitant_reply': [{'corrected_english': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm 
here to help you improve your English and discuss various topics. I see you're interested in Science. That's a 
fascinating field! What particular area of Science are you interested in?", 'corrected_native': 'Привет, Джо! 
Приятно познакомиться. Я - ИИ, разработанный OpenAI. Я здесь, чтобы помочь вам улучшить ваш английский и обсудить 
различные темы. Я вижу, что вы интересуетесь наукой. Это увлекательная область! В какой конкретной области науки вы
заинтересованы?', 'model_english_reply': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm here
to help you improve your English and discuss various topics. I see you're interested in Science. That's a 
fascinating field! What particular area of Science are you interested in?", 'model_native_reply': 'Привет, Джо! 
Приятно познакомиться. Я - ИИ, разработанный OpenAI. Я здесь, чтобы помочь вам улучшить ваш английский и обсудить 
различные темы. Я вижу, что вы интересуетесь наукой. Это увлекательная область! В какой конкретной области науки вы
заинтересованы?'}]}

Input: I am interested in biology

{'assitant_reply': [{'corrected_english': "That's great, Joe! Biology is a vast and diverse field, encompassing 
everything from the study of microscopic organisms to the complex ecosystems of our planet. Do you have a specific 
topic in biology that you're particularly interested in?", 'corrected_native': 'Это здорово, Джо! Биология - это 
обширная и разнообразная область, охватывающая все, от изучения микроскопических организмов до сложных экосистем 
нашей планеты. Есть ли у вас конкретная тема в биологии, которая вас особенно интересует?', 'model_english_reply': 
"That's great, Joe! Biology is a vast and diverse field, encompassing everything from the study of microscopic 
organisms to the complex ecosystems of our planet. Do you have a specific topic in biology that you're particularly
interested in?", 'model_native_reply': 'Это здорово, Джо! Биология - это обширная и разнообразная область, 
охватывающая все, от изучения микроскопических организмов до сложных экосистем нашей планеты. Есть ли у вас 
конкретная тема в биологии, которая вас особенно интересует?'}]}

Input: I am loving what charles darwin doing in his work

...

{'assitant_reply': [{'corrected_english': "Indeed, Darwin's theory of evolution is a cornerstone of modern biology.
It explains how species adapt and change over time through the process of natural selection. Is there a particular 
aspect of the theory that you find most intriguing?", 'corrected_native': 'Действительно, теория Дарвина об 
эволюции является краеугольным камнем современной биологии. Она объясняет, как виды адаптируются и меняются со 
временем через процесс естественного отбора. Есть ли какой-то конкретный аспект теории, который вас больше всего 
интригует?', 'model_english_reply': "Indeed, Darwin's theory of evolution is a cornerstone of modern biology. It 
explains how species adapt and change over time through the process of natural selection. Is there a particular 
aspect of the theory that you find most intriguing?", 'model_native_reply': 'Действительно, теория Дарвина об 
эволюции является краеугольным камнем современной биологии. Она объясняет, как виды адаптируются и меняются со 
временем через процесс естественного отбора. Есть ли какой-то конкретный аспект теории, который вас больше всего 
интригует?'}]}

Input: Is how the fittest is chosen

{'assitant_reply': [{'corrected_english': "Ah, you're referring to the concept of 'survival of the fittest.' This 
is a key part of Darwin's theory of natural selection. It suggests that individuals who are best adapted to their 
environment are more likely to survive and reproduce, passing on their advantageous traits to their offspring. Over
time, these traits become more common in the population, leading to evolutionary change. It's a fascinating 
process!", 'corrected_native': "А, вы говорите о концепции 'выживания самого приспособленного'. Это ключевая часть 
теории Дарвина о естественном отборе. Она предполагает, что особи, которые лучше всего приспособлены к своей среде,
более вероятно выживут и размножатся, передавая свои преимущественные черты своему потомству. Со временем эти черты
становятся более распространенными в популяции, что приводит к эволюционным изменениям. Это увлекательный 
процесс!", 'model_english_reply': "Ah, you're referring to the concept of 'survival of the fittest.' This is a key 
part of Darwin's theory of natural selection. It suggests that individuals who are best adapted to their 
environment are more likely to survive and reproduce, passing on their advantageous traits to their offspring. Over
time, these traits become more common in the population, leading to evolutionary change. It's a fascinating 
process!", 'model_native_reply': "А, вы говорите о концепции 'выживания самого приспособленного'. Это ключевая 
часть теории Дарвина о естественном отборе. Она предполагает, что особи, которые лучше всего приспособлены к своей 
среде, более вероятно выживут и размножатся, передавая свои преимущественные черты своему потомству. Со временем 
эти черты становятся более распространенными в популяции, что приводит к эволюционным изменениям. Это увлекательный
процесс!"}]}

Input: How do the fittest is chosen? keep it short pls

{'assitant_reply': [{'corrected_english': "In natural selection, the 'fittest' are those who are best adapted to 
their environment. They survive, reproduce, and pass on their traits to the next generation.", 'corrected_native': 
"В естественном отборе 'самые приспособленные' - это те, кто лучше всего приспособлен к своей среде. Они выживают, 
размножаются и передают свои черты следующему поколению.", 'model_english_reply': "In natural selection, the 
'fittest' are those who are best adapted to their environment. They survive, reproduce, and pass on their traits to
the next generation.", 'model_native_reply': "В естественном отборе 'самые приспособленные' - это те, кто лучше 
всего приспособлен к своей среде. Они выживают, размножаются и передают свои черты следующему поколению."}]}

In [14]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: Here's a brief background about me:  ```My name is Joe. I am 23 years old.  My ethnicity is Asian and my 
native language is Russian.  I work as a Software Engineer. One of my interests is Science.   Remember my knowledge
in English is in the Advanced level.  Start by introducing yourself to me and I would like to talk about 
Science.```

{'assitant_reply': [{'corrected_english': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm 
here to help you improve your English and discuss various topics. I see you're interested in Science. That's a 
fascinating field! What particular area of Science are you interested in?", 'corrected_native': 'Привет, Джо! 
Приятно познакомиться. Я - ИИ, разработанный OpenAI. Я здесь, чтобы помочь вам улучшить ваш английский и обсудить 
различные темы. Я вижу, что вы интересуетесь наукой. Это увлекательная область! В какой конкретной области науки вы
заинтересованы?', 'model_english_reply': "Hello Joe! It's nice to meet you. I'm an AI developed by OpenAI. I'm here
to help you improve your English and discuss various topics. I see you're interested in Science. That's a 
fascinating field! What particular area of Science are you interested in?", 'model_native_reply': 'Привет, Джо! 
Приятно познакомиться. Я - ИИ, разработанный OpenAI. Я здесь, чтобы помочь вам улучшить ваш английский и обсудить 
различные темы. Я вижу, что вы интересуетесь наукой. Это увлекательная область! В какой конкретной области науки вы
заинтересованы?'}]}

Input: I am interested in astronomy

{'assitant_reply': [{'corrected_english': "That's wonderful, Joe! Astronomy is a fascinating field that explores 
the mysteries of the universe. Are there any specific topics in astronomy that you're particularly interested in?",
'corrected_native': 'Это замечательно, Джо! Астрономия - это увлекательная область, которая исследует тайны 
вселенной. Есть ли какие-то конкретные темы в астрономии, которые вас особенно интересуют?', 'model_english_reply':
"That's wonderful, Joe! Astronomy is a fascinating field that explores the mysteries of the universe. Are there any
specific topics in astronomy that you're particularly interested in?", 'model_native_reply': 'Это замечательно, 
Джо! Астрономия - это увлекательная область, которая исследует тайны вселенной. Есть ли какие-то конкретные темы в 
астрономии, которые вас особенно интересуют?'}]}

Input: I am interested in the field of cosmology

...

{'assitant_reply': [{'corrected_english': "The Big Bang theory is a central concept in cosmology. It's the idea 
that the universe began as a hot, dense point nearly 14 billion years ago. This theory is supported by various 
observations, including the redshift of distant galaxies and the existence of cosmic microwave background 
radiation. Do you have any specific questions about the Big Bang theory?", 'corrected_native': 'Теория Большого 
взрыва - это центральная концепция в космологии. Это идея, что вселенная началась как горячая, плотная точка почти 
14 миллиардов лет назад. Эту теорию поддерживают различные наблюдения, включая красное смещение далеких галактик и 
существование космического микроволнового фонового излучения. У вас есть какие-то конкретные вопросы о теории 
Большого взрыва?', 'model_english_reply': "The Big Bang theory is a central concept in cosmology. It's the idea 
that the universe began as a hot, dense point nearly 14 billion years ago. This theory is supported by various 
observations, including the redshift of distant galaxies and the existence of cosmic microwave background 
radiation. Do you have any specific questions about the Big Bang theory?", 'model_native_reply': 'Теория Большого 
взрыва - это центральная концепция в космологии. Это идея, что вселенная началась как горячая, плотная точка почти 
14 миллиардов лет назад. Эту теорию поддерживают различные наблюдения, включая красное смещение далеких галактик и 
существование космического микроволнового фонового излучения. У вас есть какие-то конкретные вопросы о теории 
Большого взрыва?'}]}

Input: How those it begins keep it short.

{'assitant_reply': [{'corrected_english': 'The Big Bang theory suggests that the universe began as a singularity, a
point of infinite density and temperature, which then expanded and cooled to form the universe we see today.', 
'corrected_native': 'Теория Большого взрыва предполагает, что вселенная началась как сингулярность, точка 
бесконечной плотности и температуры, которая затем расширилась и охладилась, образуя вселенную, которую мы видим 
сегодня.', 'model_english_reply': 'The Big Bang theory suggests that the universe began as a singularity, a point 
of infinite density and temperature, which then expanded and cooled to form the universe we see today.', 
'model_native_reply': 'Теория Большого взрыва предполагает, что вселенная началась как сингулярность, точка 
бесконечной плотности и температуры, которая затем расширилась и охладилась, образуя вселенную, которую мы видим 
сегодня.'}]}

Input: What does the roman catholic reaction to big bang theory keep it short

{'assitant_reply': [{'corrected_english': 'The Roman Catholic Church generally accepts the Big Bang theory, seeing 
no conflict between it and the concept of a divine Creator.', 'corrected_native': 'Римско-католическая церковь в 
целом принимает теорию Большого взрыва, не видя в ней противоречия концепции божественного Создателя.', 
'model_english_reply': 'The Roman Catholic Church generally accepts the Big Bang theory, seeing no conflict between
it and the concept of a divine Creator.', 'model_native_reply': 'Римско-католическая церковь в целом принимает 
теорию Большого взрыва, не видя в ней противоречия концепции божественного Создателя.'}]}

In [15]:
for i in range(6):
    user_input = input('Prompt:')
    print(f'Input: {user_input}')
    response=chatbot(user_input)
    message_objects.append({"role":"assistant","content":str(response)})
    print(f"{message_objects[int(len(message_objects)-1)]['content']}\n")

Input: